In [ ]:
# 3-PARAMETER SIDE_BY_SIDE MODEL 
NAME = "3param-dbwd"

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(16 * 197 * 98, 256)  # Adjusted input size
        self.fc2 = nn.Linear(256, 3) 

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x


In [ ]:
# DB for DOUBLE WIDE IMAGE
class CustomDataset(Dataset):
    def __init__(self, image_files):
        self.image_files = image_files
        self.pattern = r"epsnx([\d.-]+)_alfax([\d.-]+)_betax([\d.-]+)_epsny([\d.-]+)_alfay([\d.-]+)_betay([\d.-]+)_epsnz([\d.-]+)_alfaz([\d.-]+)_betaz([\d.-]+)_*[\d]*\.png"

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, index):
        image_path = self.image_files[index]
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (394, 197)).astype(np.float32) / 255.0

        # add dummy dim (1, x, x)
        image = np.expand_dims(image, axis=0) 

        image_name = os.path.basename(image_path)
        matches = re.search(self.pattern, image_name)
        if matches is None:
            print(f"No match found for file: {image_name}")
            return None
        variables = [float(matches.group(i)) for i in range(1, 10) if matches.group(i)]
        epsnx = variables[0]  # epsnx value
        alfa_x = variables[1]  # alfax value
        beta_x = variables[2]  # betax value
        return image, torch.tensor([epsnx, alfa_x, beta_x])  # Returns epsnx, alfa x and betax as the labels


In [ ]:
# DB for SINGLE WIDE IMAGE
class CustomDataset(Dataset):
    def __init__(self, image_files):
        self.image_files = image_files
        self.pattern = r"epsnx([\d.-]+)_alfax([\d.-]+)_betax([\d.-]+)_epsny([\d.-]+)_alfay([\d.-]+)_betay([\d.-]+)_epsnz([\d.-]+)_alfaz([\d.-]+)_betaz([\d.-]+)_*[\d]*\.png"

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, index):
        image_path = self.image_files[index]
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (294, 294)).astype(np.float32) / 255.0

        # add dummy dim (1, x, x)
        image = np.expand_dims(image, axis=0) 

        image_name = os.path.basename(image_path)
        matches = re.search(self.pattern, image_name)
        if matches is None:
            print(f"No match found for file: {image_name}")
            return None
        variables = [float(matches.group(i)) for i in range(1, 10) if matches.group(i)]
        epsnx = variables[0]  # epsnx value
        alfa_x = variables[1]  # alfax value
        beta_x = variables[2]  # betax value
        return image, torch.tensor([epsnx, alfa_x, beta_x])  # Returns epsnx, alfa x and betax as the labels


In [ ]:
# 3_PARAM MODEL
NAME = "3param-2xeverything-bw"

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(16 * 147 * 147, 256)  # Adjust this if needed
        self.fc2 = nn.Linear(256, 3)  # Now predicting three variables

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x



In [ ]:
# 2_PARAM SIMPLE MODEL SINGLE IMAGE
NAME = "2param-bw-sgwd"

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)  # Input channel changed to 1
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(16 * 147 * 147, 256)  # Adjust this if needed
        self.fc2 = nn.Linear(256, 2)  # Now predicting three variables

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x



In [ ]:
# 2_PARAM DATASET
class CustomDataset(Dataset):
    def __init__(self, image_dir):
        self.image_dir = image_dir
        self.image_files = sorted(os.listdir(image_dir))
        self.pattern = r"epsnx([\d.-]+)_alfax([\d.-]+)_betax([\d.-]+)_epsny([\d.-]+)_alfay([\d.-]+)_betay([\d.-]+)_epsnz([\d.-]+)_alfaz([\d.-]+)_betaz([\d.-]+)_*[\d]*\.png"

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, index):
        image_name = self.image_files[index]
        image_path = os.path.join(self.image_dir, image_name)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Reading image as grayscale
        #
        image = cv2.resize(image, (294, 294))
        #
        matches = re.search(self.pattern, image_name)
        if matches is None:
            print(f"No match found for file: {image_name}")
            return None
        variables = [float(matches.group(i)) for i in range(1, 10) if matches.group(i)]
        alfa_x = variables[1]  # alfax value
        beta_x = variables[2]  # betax value
        return image, torch.tensor([alfa_x, beta_x])  # Returns epsnx, alfa x and betax as the labels

In [ ]:
# 3_PARAM DATASET
class CustomDataset(Dataset):
    def __init__(self, image_dir):
        self.image_dir = image_dir
        self.image_files = sorted(os.listdir(image_dir))
        self.pattern = r"epsnx([\d.-]+)_alfax([\d.-]+)_betax([\d.-]+)_epsny([\d.-]+)_alfay([\d.-]+)_betay([\d.-]+)_epsnz([\d.-]+)_alfaz([\d.-]+)_betaz([\d.-]+)_*[\d]*\.png"

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, index):
        image_name = self.image_files[index]
        image_path = os.path.join(self.image_dir, image_name)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        #
        image = cv2.resize(image, (294, 294))
        #
        matches = re.search(self.pattern, image_name)
        if matches is None:
            print(f"No match found for file: {image_name}")
            return None
        variables = [float(matches.group(i)) for i in range(1, 10) if matches.group(i)]
        epsnx = variables[0]  # epsnx value
        alfa_x = variables[1]  # alfax value
        beta_x = variables[2]  # betax value
        return image, torch.tensor([epsnx, alfa_x, beta_x])  # Returns epsnx, alfa x and betax as the labels

In [ ]:
# 6_PARAM DATASET
class CustomDataset(Dataset):
    def __init__(self, image_dir):
        self.image_dir = image_dir
        self.image_files = sorted(os.listdir(image_dir))
        self.pattern = r"epsnx([\d.-]+)_alfax([\d.-]+)_betax([\d.-]+)_epsny([\d.-]+)_alfay([\d.-]+)_betay([\d.-]+)_epsnz([\d.-]+)_alfaz([\d.-]+)_betaz([\d.-]+)_*[\d]*\.png"

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, index):
        image_name = self.image_files[index]
        image_path = os.path.join(self.image_dir, image_name)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        #
        image = cv2.resize(image, (294, 294))
        #
        matches = re.search(self.pattern, image_name)
        if matches is None:
            print(f"No match found for file: {image_name}")
            return None
        variables = [float(matches.group(i)) for i in range(1, 10) if matches.group(i)]
        epsnx = variables[0]  # epsnx value
        alfa_x = variables[1]  # alfax value
        beta_x = variables[2]  # betax value
        epsny = variables[3]  # epsnx value
        alfa_y = variables[4]  # alfax value
        beta_y = variables[5]  # betax value
        return image, torch.tensor([epsnx, alfa_x, beta_x, epsny, alfa_y, beta_y])  # Returns epsnx, alfa x and betax as the labels

In [ ]:
# for doublewides
def __getitem__(self, index):
        image_name = self.image_files[index]
        image_path = os.path.join(self.image_dir, image_name)
        image = cv2.imread(image_path)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (394, 197))
        matches = re.search(self.pattern, image_name)
        variables = [float(matches.group(i)) for i in range(1, 10) if matches.group(i)]
        emm_x = variables[0] # alfax value
        alfa_x = variables[1] # alfax value
        beta_x = variables[2] # betax value
        return image, torch.tensor([emm_x, alfa_x, beta_x]) # returns alfa x and betax as the labels